In [ ]:
import glob
import astropy.coordinates as coord
from astropy.io import fits, ascii
from astropy.table import Table, vstack
import astropy.units as u
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from pyia import GaiaData
import gala.coordinates as gc
from scipy.ndimage import gaussian_filter
from scipy.stats import binned_statistic_2d
from gala.mpl_style import hesperia, laguna, center_emph, center_deemph
from tqdm import tqdm
import dustmaps
from dustmaps.sfd import SFDQuery

sfd = SFDQuery()

In [ ]:
L_lim = [-40, 40] * u.deg
B_lim = [-30, 30] * u.deg

In [ ]:
import scipy.stats

def get_ext(G, bp, rp, ebv, maxnit=10):
    """ Compute the Gaia extinctions assuming relations from Babusieux
    Arguments: G, bp, rp, E(B-V)
    maxnit -- number of iterations
    Returns extinction in G,bp, rp
    Author: Sergey Koposov skoposov@cmu.edu
    """
    c1, c2, c3, c4, c5, c6, c7 = [0.9761, -0.1704,
                                  0.0086, 0.0011, -0.0438, 0.0013, 0.0099]
    d1, d2, d3, d4, d5, d6, d7 = [
        1.1517, -0.0871, -0.0333, 0.0173, -0.0230, 0.0006, 0.0043]
    e1, e2, e3, e4, e5, e6, e7 = [
        0.6104, -0.0170, -0.0026, -0.0017, -0.0078, 0.00005, 0.0006]
    A0 = 3.1*ebv
    P1 = np.poly1d([c1, c2, c3, c4][::-1])

    def F1(bprp): return np.poly1d(
        [c1, c2, c3, c4][::-1])(bprp)+c5*A0+c6*A0**2+c7*bprp*A0

    def F2(bprp): return np.poly1d(
        [d1, d2, d3, d4][::-1])(bprp)+d5*A0+d6*A0**2+d7*bprp*A0

    def F3(bprp): return np.poly1d(
        [e1, e2, e3, e4][::-1])(bprp)+e5*A0+e6*A0**2+e7*bprp*A0
    xind = np.isfinite(bp+rp+G)
    curbp = bp-rp
    for i in range(maxnit):
        AG = F1(curbp)*A0
        Abp = F2(curbp)*A0
        Arp = F3(curbp)*A0
        curbp1 = bp-rp-Abp+Arp
        delta = np.abs(curbp1-curbp)[xind]
        print(scipy.stats.scoreatpercentile(delta, 99))
        curbp = curbp1
    AG = F1(curbp)*A0
    Abp = F2(curbp)*A0
    Arp = F3(curbp)*A0
    return AG, Abp, Arp

In [ ]:
def reflex(c):
    c = coord.SkyCoord(c)
    # Correct for reflex motion
    v_sun = coord.Galactocentric.galcen_v_sun
    observed = c.transform_to(coord.Galactic)
    rep = observed.cartesian.without_differentials()
    rep = rep.with_differentials(observed.cartesian.differentials['s'] + v_sun)
    return coord.Galactic(rep).transform_to(c.frame)

BP-RP and G mag cuts to get ~A/B stars @ 30 kpc

In [ ]:
g = GaiaData('/Users/adrian/data/GaiaDR2/blue_distant.fits')
g = g[g.parallax < 0.5*u.mas]
len(g)

In [ ]:
c = coord.SkyCoord(ra=g.ra, dec=g.dec,
                   distance=np.full(len(g), 45)*u.kpc,
                   pm_ra_cosdec=g.pmra,
                   pm_dec=g.pmdec,
                   radial_velocity=np.zeros(len(g))*u.km/u.s)

In [ ]:
EBV = sfd(c)

In [ ]:
G = g.phot_g_mean_mag.value
bp = g.phot_bp_mean_mag.value
rp = g.phot_rp_mean_mag.value
A_G, A_BP, A_RP = get_ext(G, bp, rp, EBV)

In [ ]:
ext_mask = np.isfinite(A_G) & np.isfinite(A_BP) & np.isfinite(A_RP) & (A_G < 4)
ext_mask.sum()

In [ ]:
mag_c = c.transform_to(gc.MagellanicStream)
mag_c = reflex(mag_c)

In [ ]:
L = mag_c.L.wrap_at(180*u.deg)
B = mag_c.B
mag_mask = ((L > L_lim[0]) & (L < L_lim[1]) &
            (B > B_lim[0]) & (B < B_lim[1]))

# exclude center...
lmc_mask = (np.sqrt(L**2 + (B-4*u.deg)**2) < 7*u.deg) & (np.sqrt(L**2 + (B-4*u.deg)**2) > 2*u.deg)
smc_mask = np.sqrt((L+15*u.deg)**2 + (B+11*u.deg)**2) < 3.5*u.deg
# bridge_mask = np.sqrt((L+10*u.deg)**2 + (B+6*u.deg)**2) < 4*u.deg

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

ax.plot(mag_c.L.wrap_at(180*u.deg).degree[ext_mask & mag_mask], 
        mag_c.B.degree[ext_mask & mag_mask], 
        linestyle='none', marker='.', alpha=0.1)

ax.set_xlim(L_lim[1].value, L_lim[0].value)
ax.set_ylim(B_lim[0].value, B_lim[1].value)
ax.set_aspect('equal')

In [ ]:
bprp0 = (g.phot_bp_mean_mag - g.phot_rp_mean_mag) - (A_BP - A_RP)*u.mag
G0 = g.phot_g_mean_mag - A_G*u.mag

In [ ]:
isos = dict()
for fn in glob.glob('/Users/adrian/Downloads/*Myr*.cmd'):
    iso = Table.read(fn, 
                     format='ascii.commented_header', header_start=12)
    phasecut = (iso['phase']>=0) & (iso['phase']<3)
    iso = iso[phasecut]
    # isos[iso['[Fe/H]_init'][0]] = iso
    isos[iso['isochrone_age_yr'][0]] = iso

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

ax.plot(bprp0[smc_mask], G0[smc_mask], marker=',', linestyle='none', alpha=0.2)

for feh, iso in isos.items():
    ax.plot(iso['Gaia_BP_DR2Rev']-iso['Gaia_RP_DR2Rev'],
            iso['Gaia_G_DR2Rev'] + coord.Distance(50*u.kpc).distmod.value)

ax.set_xlim(-2, 0.5)
ax.set_ylim(20, 12)

In [ ]:
# more_mask = (G0 > 16.*u.mag) & (bprp0 > -0.5*u.mag)
# more_mask = (bprp0 < -0*u.mag) & (bprp0 > -0.5*u.mag)
more_mask = (bprp0 < 0.15*u.mag) & (bprp0 > -0.5*u.mag) & (G0 > (2.5/0.5*bprp0 + 17.5*u.mag))

fig, ax = plt.subplots(1, 1, figsize=(6, 6))

ax.plot(bprp0, G0, marker=',', linestyle='none', alpha=0.2)
ax.plot(bprp0[more_mask], G0[more_mask], marker=',', linestyle='none', alpha=1)

ax.set_xlim(-2, 0.5)
ax.set_ylim(12, 20)

In [ ]:
all_mask = mag_mask & ext_mask & more_mask #& np.logical_not(lmc_mask) & np.logical_not(smc_mask)
full_lmc_mask = mag_mask & ext_mask & more_mask & lmc_mask
full_smc_mask = mag_mask & ext_mask & more_mask & smc_mask
all_mask.sum(), full_lmc_mask.sum(), full_smc_mask.sum()

In [ ]:
plt.hist(mag_c[full_lmc_mask].pm_L_cosB, bins=np.linspace(-5, 5, 128));
plt.hist(mag_c[full_lmc_mask].pm_B, bins=np.linspace(-5, 5, 128));

In [ ]:
med_LMC_L = np.median(mag_c[full_lmc_mask].L.wrap_at(180*u.deg))
med_LMC_B = np.median(mag_c[full_lmc_mask].B)
med_LMC_pmL = np.median(mag_c[full_lmc_mask].pm_L_cosB)
med_LMC_pmB = np.median(mag_c[full_lmc_mask].pm_B)

med_SMC_pmL = np.median(mag_c[full_smc_mask].pm_L_cosB)
med_SMC_pmB = np.median(mag_c[full_smc_mask].pm_B)

In [ ]:
stat_muL = binned_statistic_2d(mag_c.L.wrap_at(180*u.deg).degree[all_mask],
                               mag_c.B.degree[all_mask],
                               mag_c.pm_L_cosB[all_mask] - med_LMC_pmL,
                               bins=(np.arange(L_lim[0].value, L_lim[1].value+0.1, 0.75),
                                     np.arange(B_lim[0].value, B_lim[1].value+0.1, 0.75)),
                               statistic=np.nanmean)

In [ ]:
derp = stat_muL.statistic.ravel()
plt.hist(derp[np.isfinite(derp)], bins=np.linspace(-2, 2, 128));

In [ ]:
from astropy.convolution import Gaussian2DKernel, convolve
gauss = Gaussian2DKernel(0.5)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 7))

H = stat_muL.statistic.T
# H = convolve(H, gauss)
cc = ax.pcolormesh(stat_muL.x_edge, stat_muL.y_edge, H,
                   vmin=-1.25, vmax=0.5, cmap='magma')
cb = fig.colorbar(cc)
cb.set_label(r'${\rm mean}(\mu_L \cos B)$')

ax.set_xlim(L_lim[1].value, L_lim[0].value)
ax.set_ylim(B_lim[0].value, B_lim[1].value)
ax.set_aspect('equal')

ax.set_xlabel(r'$L_{\rm MS}$ [deg]')
ax.set_ylabel(r'$B_{\rm MS}$ [deg]')

fig.tight_layout()

In [ ]:
pm_bins = np.arange(-5, 5+0.1, 0.1)

fg_pm, pm_x, pm_y = np.histogram2d(mag_c.pm_L_cosB[all_mask].value, 
                                   mag_c.pm_B[all_mask].value,
                                   bins=(pm_bins, pm_bins))

fig, ax = plt.subplots(1, 1, figsize=(5, 5))

ax.pcolormesh(pm_x, pm_y, fg_pm.T)

r = mpl.patches.Circle((med_LMC_pmL.value, med_LMC_pmB.value), radius=0.8, facecolor='none', edgecolor='k')
ax.add_patch(r)

r = mpl.patches.Circle((med_SMC_pmL.value, med_SMC_pmB.value), radius=0.6, facecolor='none', edgecolor='k')
ax.add_patch(r)

fig.tight_layout()

In [ ]:
pm_mask = np.sqrt((mag_c.pm_L_cosB - med_LMC_pmL)**2 + 
                  (mag_c.pm_B - med_LMC_pmB)**2) < 0.8*u.mas/u.yr
pm_mask |= np.sqrt((mag_c.pm_L_cosB - med_SMC_pmL)**2 + 
                   (mag_c.pm_B - med_SMC_pmB)**2) < 0.6*u.mas/u.yr

In [ ]:
stat_muL = binned_statistic_2d(mag_c.L.wrap_at(180*u.deg).degree[all_mask & pm_mask],
                               mag_c.B.degree[all_mask & pm_mask],
                               mag_c.pm_L_cosB[all_mask & pm_mask] - med_LMC_pmL,
                               bins=(np.arange(L_lim[0].value, L_lim[1].value+0.1, 0.5),
                                     np.arange(B_lim[0].value, B_lim[1].value+0.1, 0.5)),
                               statistic=np.nanmean)

In [ ]:
stat_muB = binned_statistic_2d(mag_c.L.wrap_at(180*u.deg).degree[all_mask & pm_mask],
                               mag_c.B.degree[all_mask & pm_mask],
                               mag_c.pm_B[all_mask & pm_mask] - med_LMC_pmB,
                               bins=(np.arange(L_lim[0].value, L_lim[1].value+0.1, 0.5),
                                     np.arange(B_lim[0].value, B_lim[1].value+0.1, 0.5)),
                               statistic=np.nanmean)

In [ ]:
# derp = stat_muL.statistic.ravel()
derp = stat_muB.statistic.ravel()
plt.hist(derp[np.isfinite(derp)], bins=np.linspace(-1, 1, 64));

In [ ]:
cmap = plt.get_cmap('center_deemph')
cmap.set_bad('k',1.0)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 7))

# H = stat_muL.statistic.T
H = stat_muB.statistic.T

H = np.ma.array(H, mask=np.isnan(H)) 

# H = convolve(H, gauss)
# cc = ax.pcolormesh(stat_muL.x_edge, stat_muL.y_edge, H,
#                    vmin=-0.6, vmax=0.3, cmap='rainbow')
cc = ax.pcolormesh(stat_muB.x_edge, stat_muB.y_edge, H,
                   vmin=-0.35, vmax=0.35, cmap=cmap)
cb = fig.colorbar(cc)
cb.set_label(r'${\rm mean}(\mu_B)$')

ax.set_xlim(30, -40)
ax.set_ylim(-30, 30)
ax.set_aspect('equal')

ax.set_xlabel(r'$L_{\rm MS}$ [deg]')
ax.set_ylabel(r'$B_{\rm MS}$ [deg]')

fig.tight_layout()

fig.set_facecolor('w')

In [ ]:
dL = mag_c.L.wrap_at(180*u.deg)[all_mask & pm_mask] - med_LMC_L.wrap_at(180*u.deg)
dB = mag_c.B[all_mask & pm_mask] - med_LMC_B
dpmL = mag_c.pm_L_cosB[all_mask & pm_mask] - med_LMC_pmL
dpmB = mag_c.pm_B[all_mask & pm_mask] - med_LMC_pmB

In [ ]:
# TOTAL HACK:
dist = 50 * u.kpc
x = (dL * dist).to(u.kpc, u.dimensionless_angles())
y = (dB * dist).to(u.kpc, u.dimensionless_angles())
vx = (dpmL * dist).to(u.km/u.s, u.dimensionless_angles())
vy = (dpmB * dist).to(u.km/u.s, u.dimensionless_angles())

R = np.sqrt(x**2 + y**2)

vR = (x*vx + y*vy) / R
vphi = (x*vy - y*vx) / R

In [ ]:
vv = vphi

stat = binned_statistic_2d(mag_c.L.wrap_at(180*u.deg).degree[all_mask & pm_mask],
                           mag_c.B.degree[all_mask & pm_mask],
                           vv.value,
                           bins=(np.arange(L_lim[0].value, L_lim[1].value+0.1, 1),
                                 np.arange(B_lim[0].value, B_lim[1].value+0.1, 1)),
                           statistic=np.nanmean)

fig, ax = plt.subplots(1, 1, figsize=(15, 7))

H = stat.statistic.T
H = np.ma.array(H, mask=np.isnan(H)) 

# H = convolve(H, gauss)
# cc = ax.pcolormesh(stat_muL.x_edge, stat_muL.y_edge, H,
#                    vmin=-0.6, vmax=0.3, cmap='rainbow')
cc = ax.pcolormesh(stat.x_edge, stat.y_edge, H,
                   vmin=np.median(vv).value-30, vmax=np.median(vv).value+30, 
                   cmap=cmap)
cb = fig.colorbar(cc)
cb.set_label(r'${\rm mean}(\mu_B)$')

ax.set_xlim(30, -40)
ax.set_ylim(-30, 30)
ax.set_aspect('equal')

ax.set_xlabel(r'$L_{\rm MS}$ [deg]')
ax.set_ylabel(r'$B_{\rm MS}$ [deg]')

fig.tight_layout()

fig.set_facecolor('w')

In [ ]:
vv = vphi

stat = binned_statistic_2d(mag_c.L.wrap_at(180*u.deg).degree[all_mask & pm_mask],
                           mag_c.B.degree[all_mask & pm_mask],
                           vv.value,
                           bins=(np.arange(L_lim[0].value, L_lim[1].value+0.1, 1),
                                 np.arange(B_lim[0].value, B_lim[1].value+0.1, 1)),
                           statistic=np.nanmean)

fig, ax = plt.subplots(1, 1, figsize=(15, 7))

H = stat.statistic.T
H = np.ma.array(H, mask=np.isnan(H)) 

# H = convolve(H, gauss)
cc = ax.pcolormesh(stat.x_edge, stat.y_edge, H,
                   vmin=-20, vmax=80, 
                   cmap=cmap)
cb = fig.colorbar(cc)
cb.set_label(r'${\rm mean}(\mu_B)$')

ax.set_xlim(30, -40)
ax.set_ylim(-30, 30)
ax.set_aspect('equal')

ax.set_xlabel(r'$L_{\rm MS}$ [deg]')
ax.set_ylabel(r'$B_{\rm MS}$ [deg]')

fig.tight_layout()

fig.set_facecolor('w')

In [ ]:
ddx = (med_LMC_pmL * 50*u.Myr).to(u.deg).value
ddy = (med_LMC_pmB * 50*u.Myr).to(u.deg).value

In [ ]:
plt.arrow(med_LMC_L.degree, med_LMC_B.degree,
          ddx, ddy, linewidth=2, color='r')
plt.xlim(-10, 10)
plt.ylim(-10, 10)